### Create gold tables
1. movies_rated
2. person
3. title_person
4. rating_metrics


In [0]:
USE CATALOG imdb_dev;

In [0]:
SELECT current_catalog();

##### Create table movies_rated

In [0]:
CREATE OR REPLACE TABLE imdb_dev.gold.title_rating AS
SELECT 
       mv.tconst,
       mv.primary_title,
       mv.original_title,
       mv.start_year AS release_year,
       mv.runtime_minutes,
       mv.genres,
       COALESCE(tr.average_rating, 0.0) AS average_rating,
       COALESCE(tr.num_votes, 0) AS num_votes
FROM imdb_dev.silver.movies mv
LEFT JOIN imdb_dev.silver.title_ratings tr
ON mv.tconst = tr.tconst;

In [0]:
select * from imdb_dev.gold.title_rating;

##### Create table person

In [0]:
CREATE OR REPLACE TABLE imdb_dev.gold.person AS
SELECT DISTINCT
  nconst,
  primary_name,
  primary_profession,
  birth_year,
  death_year
FROM imdb_dev.silver.name_basics;

##### Create table person_role

In [0]:
-- Use only relavant roles ('director','writer','actor','actress','producer')
CREATE OR REPLACE TABLE imdb_dev.gold.person_role AS
SELECT DISTINCT
  p.tconst,
  p.nconst,
  LOWER(p.category) AS role,
  p.ordering
FROM imdb_dev.silver.title_principals p
WHERE p.category IN ('director','writer','actor','actress','producer');

##### Create table title_person

In [0]:
CREATE OR REPLACE TABLE imdb_dev.gold.title_person AS
SELECT tr.primary_title, 
       tr.release_year, 
       tr.genres,
       pr.role, 
       p.primary_name, 
       tr.average_rating, 
       tr.num_votes
FROM imdb_dev.gold.title_rating tr
LEFT JOIN imdb_dev.gold.person_role pr USING (tconst)
LEFT JOIN imdb_dev.gold.person p USING (nconst);

##### Create table rating_metrics

In [0]:
CREATE OR REPLACE TABLE imdb_dev.gold.fact_title_rating AS
SELECT
  tconst,
  COALESCE(average_rating, 0.0) AS average_rating,
  COALESCE(num_votes, 0) AS num_votes
FROM imdb_dev.silver.title_ratings;